# CSCI 3155: Assignment 5

__Name__: Julia Sanford

In [1]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"Tests Passed (1 point)")
    else print(s"*** Tests Passed ($points points) ***")
}

defined function passed

## Problem 1 (10 points): Case Statements in Lettuce.

Let us take a stripped down  subset of the Lettuce language given by the grammar here but 
add a new case match statement.

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Ident(\mathbf{Identifier}) \\
& \rightarrow & Const(\mathbf{Double}) \\
& \rightarrow & True \\
& \rightarrow & False \\
& \rightarrow & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
& \rightarrow & Geq(\mathbf{Expr}, \mathbf{Expr}) \\
& \rightarrow & Not(\mathbf{Expr}) \\
& \rightarrow & IfThenElse (\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) \\
& \rightarrow & CaseMatch(\mathbf{Expr}, \mathbf{Cases}*, \mathbf{Expr}) & \text{First argument is the caseMatch expression} \\
& & & \text{followed by zero or more cases (list) and last argument is the default} \\
& \rightarrow & Let(\mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) \\[5pt]
\mathbf{Cases} & \rightarrow & SingleCase( \mathbf{Expr}, \mathbf{Expr} ) & \text{First argument is what we try to match on}\\
&& & \text{if the match succeeds, value equals second argument} \\
\end{array} $$

The value types are

- Double precision numbers in scala that are currently taken to be reals : $\mathbb{R}$.
- Booleans true/false: $\mathbb{B}$.
- Error: $\mathbf{error}$.

The semantics of `Const, True, False, Plus, Geq, And, Not, IfThenElse` and `Let` are as we
covered in the lecture (see notebook describing Lettuce operational semantics and interpreter).

We extend our semantics to `CaseMatch`. The grammar has been extended with these statements already.

A `CaseMatch` statement: 

~~~
caseMatch(e) {
   case e1 => f1
   case e2 => f2
    ...
   case ek => fk
   default => f
}
~~~

involves matching expression `e`, and cases `(e1, f1)`, ..., `(ek,fk)`, and a default `f`.

This will be represented as 
~~~
CaseMatch(e, listofCases, f)
~~~
where `listOfCases` will be
~~~
SingleCase(e1, f1), ..., SingleCase(ek,fk)
~~~

It translates into a nested `if-then-elseif-else` statement as

~~~
if (e == e1)  f1
elseif (e == e2) f2
elseif ...
elseif (e == ek) fk
else f
~~~

**Note**: By this description, and unlike in Scala, a variable used in a pattern should simply read the value of that variable instead of declaring a new one.

Extend the `evalExpr` function below with support for the `CaseMatch` statement


In [2]:
// Definitions that you will need. Make sure to execute this cell
sealed trait Program
sealed trait Expr
sealed trait Cases

case class TopLevel(e: Expr) extends Program

case class Const(f: Double) extends Expr
case class Identifier(s: String) extends Expr
case object True extends Expr
case object False extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class  Geq(e1: Expr, e2: Expr) extends Expr
case class Not(e: Expr) extends Expr
case class IfThenElse(e: Expr, eThen: Expr, eElse: Expr)  extends Expr
case class CaseMatch(e: Expr, cases: List[Cases], default: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr

case class SingleCase(e: Expr, f: Expr) extends Cases

// Values

sealed trait Value
case class NumValue(f: Double) extends Value
case class BoolValue(b: Boolean) extends Value

def valueToNum(v: Value) = v match {
    case NumValue(f) => f
    case _ => throw new IllegalArgumentException("Expected numerical value.")
}

def valueToBool(v: Value) = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException("Expected boolean value")
    
}

def valueEquals(v1: Value, v2: Value ): Boolean = (v1, v2) match {
    case (NumValue(f1), NumValue(f2)) => f1 == f2
    case (BoolValue(b1), BoolValue(b2)) => b1 == b2
    case _ => false
}


defined trait Program
defined trait Expr
defined trait Cases
defined class TopLevel
defined class Const
defined class Identifier
defined object True
defined object False
defined class Plus
defined class Geq
defined class Not
defined class IfThenElse
defined class CaseMatch
defined class Let
defined class SingleCase
defined trait Value
defined class NumValue
defined class BoolValue
defined function valueToNum
defined function valueToBool
defined function valueEquals

In [7]:
def evalExpr(e: Expr, env: Map[String, Value]): Value = {
    e match {
        case Const(f) => NumValue(f) 
        case True => BoolValue(true)
        case False => BoolValue(false)
        case Identifier(s) =>  if (env contains s) {
            env(s)
        } else {
            throw new IllegalArgumentException("Could not find variable "+ s)
        }                              
        case Plus(e1, e2) => {
            val v1 = valueToNum(evalExpr(e1, env))
            val v2 = valueToNum(evalExpr(e2, env))
            NumValue(v1 + v2)
        }
        case Geq(e1, e2) => {
            val v1 = valueToNum(evalExpr(e1, env))
            val v2 = valueToNum(evalExpr(e2, env))
            BoolValue(v1 >= v2)
        }
        case Not(e) => {
            val v = valueToBool(evalExpr(e, env))
            BoolValue(!v)
        }
        case IfThenElse(e, e1, e2) => {
            val v1 = valueToBool(evalExpr(e, env))
            if (v1)
                evalExpr(e1, env)
            else
                evalExpr(e2, env)
        }
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)
            val newEnv = env + (x -> v1)
            evalExpr(e2, newEnv)
        }
        case CaseMatch(e, cases, f) => {
            // YOUR CODE HERE
            val value = evalExpr(e, env)
            val expr = cases.foldLeft(f) { (acc, element) => 
                element match {
                    case SingleCase(ei, fi) => {
                        if(valueEquals(value, evalExpr(ei, env))) {
                            fi
                        }
                        else {
                            acc
                        }
                    }
                }
            }
            evalExpr(expr, env)
        } 
    }

}

defined function evalExpr

In [8]:
// BEGIN TEST

// caseMatch(1)
//   case 1 => 5
//   default => 6

val caseList = List(SingleCase(Const(1.0), Const(5.0)))
val overallExpr = CaseMatch(Const(1.0), caseList, Const(6.0))

assert(evalExpr(overallExpr, Map()) == NumValue(5.0), "Test 1 Failed")

passed(3)
// END TEST

*** Tests Passed (3 points) ***

caseList: List[SingleCase] = List(SingleCase(Const(1.0),Const(5.0)))
overallExpr: CaseMatch = CaseMatch(Const(1.0),List(SingleCase(Const(1.0),Const(5.0))),Const(6.0))

In [9]:
// BEGIN TEST

// let x = 10 in 
// let y = 15 in 
// caseMatch(x + y)
//   case y+15 => y
//   case x + x => x
//   case 25 => 0
//   default => -100

val x = Identifier("x")
val y = Identifier("y")
val caseList = List( SingleCase(Plus(y, Const(15)), y), SingleCase(Plus(x, x), x), SingleCase(Const(25), Const(0)))
val caseExpr = CaseMatch(Plus(x,y), caseList, Const(-100.0))
val overallExpr: Expr = Let("x", Const(10), Let("y", Const(15), caseExpr))

assert(evalExpr(overallExpr, Map()) == NumValue(0.0), "Test 2 Failed")

passed(2)
// END TEST

*** Tests Passed (2 points) ***

x: Identifier = Identifier("x")
y: Identifier = Identifier("y")
caseList: List[SingleCase] = List(
  SingleCase(Plus(Identifier(y),Const(15.0)),Identifier(y)),
  SingleCase(Plus(Identifier(x),Identifier(x)),Identifier(x)),
  SingleCase(Const(25.0),Const(0.0))
)
caseExpr: CaseMatch = CaseMatch(Plus(Identifier(x),Identifier(y)),List(SingleCase(Plus(Identifier(y),Const(15.0)),Identifier(y)), SingleCase(Plus(Identifier(x),Identifier(x)),Identifier(x)), SingleCase(Const(25.0),Const(0.0))),Const(-100.0))
overallExpr: Expr = Let(x,Const(10.0),Let(y,Const(15.0),CaseMatch(Plus(Identifier(x),Identifier(y)),List(SingleCase(Plus(Identifier(y),Const(15.0)),Identifier(y)), SingleCase(Plus(Identifier(x),Identifier(x)),Identifier(x)), SingleCase(Const(25.0),Const(0.0))),Const(-100.0))))

In [10]:
// BEGIN TEST

/* let x = 15 in 
   let cond = caseMatch (x >= 0)
               case True: 15
               case False: 25
               default: y // Intentionally introduce an error
   in 
       cond + cond
       */

val x = Identifier("x")
val cond = Identifier("cond")
val caseList= List( SingleCase(True, Const(15)), SingleCase(False, Const(25)))
val caseExpr = Let("cond", CaseMatch(Geq(x, Const(0.0)), caseList, Identifier("y")), Plus(cond, cond))
val overallExpr2 = Let("x", Const(15), caseExpr)

assert(evalExpr(overallExpr2, Map()) == NumValue(30.0), "Test 3 failed - expected value should be 30")

passed(2)
// END TEST

*** Tests Passed (2 points) ***

x: Identifier = Identifier("x")
cond: Identifier = Identifier("cond")
caseList: List[SingleCase] = List(SingleCase(True,Const(15.0)), SingleCase(False,Const(25.0)))
caseExpr: Let = Let(cond,CaseMatch(Geq(Identifier(x),Const(0.0)),List(SingleCase(True,Const(15.0)), SingleCase(False,Const(25.0))),Identifier(y)),Plus(Identifier(cond),Identifier(cond)))
overallExpr2: Let = Let(x,Const(15.0),Let(cond,CaseMatch(Geq(Identifier(x),Const(0.0)),List(SingleCase(True,Const(15.0)), SingleCase(False,Const(25.0))),Identifier(y)),Plus(Identifier(cond),Identifier(cond))))

In [11]:
// BEGIN TEST
/* let x = 15 in 
   let cond = CaseMatch (x >= x) 
            default: x >= x + 1
   in 
        if (cond) then 20 else 40 */

val x = Identifier("x")
val cond = Identifier("cond")
val caseExpr = CaseMatch(Geq(x,x), List(), Geq(x, Plus(x, Const(1))))
val iteExpr = IfThenElse(cond, Const(20), Const(40))
val overallExpr3 = Let("x", Const(15), Let("cond", caseExpr, iteExpr))
assert(evalExpr(overallExpr3, Map()) == NumValue(40.0), "Test 4 failed - expected value should be 40")

passed(2)
// END TEST

*** Tests Passed (2 points) ***

x: Identifier = Identifier("x")
cond: Identifier = Identifier("cond")
caseExpr: CaseMatch = CaseMatch(Geq(Identifier(x),Identifier(x)),List(),Geq(Identifier(x),Plus(Identifier(x),Const(1.0))))
iteExpr: IfThenElse = IfThenElse(Identifier(cond),Const(20.0),Const(40.0))
overallExpr3: Let = Let(x,Const(15.0),Let(cond,CaseMatch(Geq(Identifier(x),Identifier(x)),List(),Geq(Identifier(x),Plus(Identifier(x),Const(1.0)))),IfThenElse(Identifier(cond),Const(20.0),Const(40.0))))

## Problem 2 (30 Points): Adding String Values

We will now extend Lettuce with support for string manipulations in addition to manipulating numbers and booleans.

In particular, we will allow the following string expressions.
- A constant string `ConstString(StringLiteral)`
- The concatenation of two strings that uses the same operator `Plus`  that was used for addition.
- Multiplication of a string with a number: `string * n`  replicates the string `n times`. This is undefined or erroneous if `n < 0` or `n` is not a whole number.
- A string length function that obtains the length of a string `StringLength(expr)`
- A substring function `SubString(expr, expr, expr)` wherein `SubString(string, i, j)` requires
first argument to be a string and the second/third argument to be whole numbers such that `0 <= i <= j < StringLength(string)`. It returns a new string that is a substring of the original string from between positions `i` and `j`. Both end points are included.

### 2 A (5 points)
Extend the grammar for Lettuce below by adding new rules corresponding to the string expressions. All string expressions must reuse the nonterminal Expr.


$$\begin{array}{rcl}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Ident(\mathbf{Identifier}) \\
& \rightarrow & Const(\mathbf{Double}) \\
& \rightarrow & True \\
& \rightarrow & False \\
& \rightarrow & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
& \rightarrow & Mult(\mathbf{Expr}, \mathbf{Expr}) \\
& \rightarrow & Geq(\mathbf{Expr}, \mathbf{Expr}) \\
& \rightarrow & Not(\mathbf{Expr}) \\
& \rightarrow & IfThenElse (\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) \\
& \rightarrow & Let(\mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) \\[5pt]
\mathbf{Double} & \rightarrow & \text{.. all scala double precision numbers ... }\\ 
\mathbf{StringLiteral} & \rightarrow & \text{... all scala strings ... } \\
\end{array} $$



$$\begin{array}{rcl}
& \rightarrow & ConstString(\mathbf{StringLiteral}) \\
& \rightarrow & StringLength(\mathbf{Expr}) \\
& \rightarrow & SubString(\mathbf{Expr, Expr, Expr}) \\
\end{array} $$

### 2 B (5 points)
Complete the scala definition for the abstract  syntax tree below by adding extra cases for supporting all the string operations in your grammar. Make sure that you use the constructor symbols `ConstString`, `StringLength`, `SubString`.

In [28]:
sealed trait Program
sealed trait Expr
sealed trait Cases

case class TopLevel(e: Expr) extends Program

case class Const(f: Double) extends Expr
case class Identifier(s: String) extends Expr
case object True extends Expr
case object False extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class  Geq(e1: Expr, e2: Expr) extends Expr
case class Not(e: Expr) extends Expr
case class IfThenElse(e: Expr, eThen: Expr, eElse: Expr)  extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr

// YOUR CODE HERE
case class ConstString(s: String) extends Expr
case class StringLength(e: Expr) extends Expr
case class SubString(e1: Expr, e2: Expr, e3: Expr) extends Expr


defined trait Program
defined trait Expr
defined trait Cases
defined class TopLevel
defined class Const
defined class Identifier
defined object True
defined object False
defined class Plus
defined class Mult
defined class Geq
defined class Not
defined class IfThenElse
defined class Let
defined class ConstString
defined class StringLength
defined class SubString

In [29]:
// BEGIN TEST
val v1 = ConstString("hello world")
val v2 = ConstString("")
val v4 = SubString(v1,Const(0), Const(3))
val v5 = StringLength(v4)
val v6 = StringLength(v5)

passed(3)
// END TEST

*** Tests Passed (3 points) ***

v1: ConstString = ConstString("hello world")
v2: ConstString = ConstString("")
v4: SubString = SubString(ConstString(hello world),Const(0.0),Const(3.0))
v5: StringLength = StringLength(SubString(ConstString(hello world),Const(0.0),Const(3.0)))
v6: StringLength = StringLength(StringLength(SubString(ConstString(hello world),Const(0.0),Const(3.0))))

In [30]:
// BEGIN TEST
val v1 = ConstString("hello world")
val v2 = ConstString("")
val v3 = Plus(v1, v2)

passed(1)
// END TEST

Tests Passed (1 point)

v1: ConstString = ConstString("hello world")
v2: ConstString = ConstString("")
v3: Plus = Plus(ConstString(hello world),ConstString())

In [31]:
// BEGIN TEST
val v1 = ConstString("hello world")
val v2 = ConstString("")
val v4 = Mult(v1, Const(20))
val v5 = Mult(v2, Const(10))

passed(1)
// END TEST

Tests Passed (1 point)

v1: ConstString = ConstString("hello world")
v2: ConstString = ConstString("")
v4: Mult = Mult(ConstString(hello world),Const(20.0))
v5: Mult = Mult(ConstString(),Const(10.0))

### 2 C (5 points)
Complete the value type `StringValue` for string values and update the functions below to handle string values.

In [32]:
sealed trait Value
case class NumValue(f: Double) extends Value
case class BoolValue(b: Boolean) extends Value
// YOUR CODE HERE
case class StringValue(s: String) extends Value

def valueToNum(v: Value): Double = v match {
    case NumValue(f) => f
    case _ => throw new IllegalArgumentException("Expected numerical value.")
}

def valueToBool(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException("Expected boolean value")
    
}

def valueToString(v: Value): String = v match {
    // YOUR CODE HERE
    case StringValue(s) => s
    case _ => throw new IllegalArgumentException("Expected string value")
}

/*--- Test if a given value is a real number --*/
def isNum(v: Value): Boolean = v match {
    // YOUR CODE HERE
    case NumValue(x) => true
    case _ => false
}


/*-- Test if a given value is a natural number 0, 1, 2, ... */
def isNatNum(v: Value): Boolean = v match {
    // YOUR CODE HERE
    case NumValue(x) if(x >= 0 && x == x.toInt) => true
    case _ => false
}

def isString(v: Value): Boolean = v match {
    // YOUR CODE HERE
    case StringValue(x) => true
    case _ => false
}


defined trait Value
defined class NumValue
defined class BoolValue
defined class StringValue
defined function valueToNum
defined function valueToBool
defined function valueToString
defined function isNum
defined function isNatNum
defined function isString

In [33]:
// BEGIN TEST
val v1 = StringValue("hello")
assert(isString(v1), "Test 1 failed")

val v2 = StringValue("world")
assert(valueToString(v2) == "world", "Test 2 failed")

passed(2)
// END TEST

*** Tests Passed (2 points) ***

v1: StringValue = StringValue("hello")
v2: StringValue = StringValue("world")

In [34]:
// BEGIN TEST
val v3 = NumValue(20.0)
assert(isNatNum(v3), "Test 3 Failed")

val v4 = NumValue(20.00000001)
assert(!isNatNum(v4), "Test 4 Failed")

val v5 = StringValue("hello"*20)
assert(isString(v5), "Test 5 passed")

assert(valueToString(v5) == "hello"*20, "Test 6 passed")

passed(3)
// END TEST

*** Tests Passed (3 points) ***

v3: NumValue = NumValue(20.0)
v4: NumValue = NumValue(20.00000001)
v5: StringValue = StringValue(
  "hellohellohellohellohellohellohellohellohellohellohellohellohellohellohellohellohellohellohellohello"
)

### 2 D (5 points)
We will now write operational semantics for multiplication that accomodates strings.

$$\begin{array}{c}
\mathbf{eval}(\texttt{e1}, \sigma) = v_1, \mathbf{eval}(\texttt{e1}, \sigma) = v_2, v_1 \in \mathbb{R}, v_2 \in \mathbb{R}\\
\hline
\mathbf{eval}(\texttt{Mult(e1, e2)}, \sigma) = v_1 \times v_2 \\
\end{array} (\text{Multiply-Numerical}) $$

Given a string $v$ and an integer $n \geq 0$, we will write $\mathbf{replicate}(v, n)$ as the string $v$
replicated $n$ times.

Complete the labelled missing portions of the operational semantic rules below (in $\color{red}{\text{red}}$). Let $\mathbb{Z}$ represent integers (a subset of reals), $\mathbb{N}$ represent natural numbers including zero and $\mathbb{Str}$ represent strings. 

$$\begin{array}{c}
\mathbf{eval}(\texttt{e1}, \sigma) = v_1, \mathbf{eval}(\texttt{e1}, \sigma) = v_2, v_1 \in \mathbb{Str}, \color{red}{1}\\
\hline
\mathbf{eval}(\texttt{Mult(e1, e2)}, \sigma) = \color{red}{2} \\
\end{array} (\text{Multiply-String-Number-OK}) $$



$$\begin{array}{c}
\mathbf{eval}(\texttt{e1}, \sigma) = v_1,  v_1 \not\in \mathbb{Str} \cup \mathbb{R}\\
\hline
\mathbf{eval}(\texttt{Mult(e1, e2)}, \sigma) = \color{red}{3} \\
\end{array} (\text{Multiply-String-Number-nok-1}) $$


$$\begin{array}{c}
\mathbf{eval}(\texttt{e1}, \sigma) = v_1,\  v_1 \in \mathbb{Str},\ \mathbf{eval}(\texttt{e2}, \sigma) = v_2,\ \ \color{red}{4} \\
\hline
\mathbf{eval}(\texttt{Mult(e1, e2)}, \sigma) = \color{red}{5} \\
\end{array} (\text{Multiply-String-Number-nok-2}) $$

Write your answer in the cell bellow. You can make a numbered list in markdown to represent your answers as follows:
1. First
2. Second
3. And so on...

$$
1)\ v_2 \in \mathbb{N} \\
2)\ v_1 * v_2 \\
3)\ errorType \\
4)\ v_2 \notin \mathbb{N} \\
5)\ errorType \\
$$

### 2 D (10 points)
Complete the implementation `evalExpr` function by modifying the existing code.

In [41]:
def evalExpr(e: Expr, env: Map[String, Value]): Value = {
    e match {
        case Const(f) => NumValue(f) 
        case True => BoolValue(true)
        case False => BoolValue(false)
        case Identifier(s) =>  if (env contains s) {
            env(s)
        } else {
            throw new IllegalArgumentException("Could not find variable "+ s)
        }        
        case ConstString(s) => { 
            // YOUR CODE HERE
            StringValue(s)
        }
        
        case StringLength(e) => {
            // YOUR CODE HERE
            NumValue(valueToString(evalExpr(e, env)).length)
        }
        
        case Plus(e1, e2) => {
            // YOUR CODE HERE
            if (isString(evalExpr(e1, env)) && isString(evalExpr(e2, env))) {
                StringValue(valueToString(evalExpr(e1, env)) + valueToString(evalExpr(e2, env)))
            }
            else if (isString(evalExpr(e1, env)) && isNum(evalExpr(e2, env))) {
                StringValue(valueToString(evalExpr(e1, env)) + valueToNum(evalExpr(e2, env)))
            }
            else if (isNum(evalExpr(e1, env)) && isString(evalExpr(e2, env))) {
                StringValue(valueToNum(evalExpr(e1, env)) + valueToString(evalExpr(e2, env)))
            }
            else {
                NumValue(valueToNum(evalExpr(e1, env)) + valueToNum(evalExpr(e2, env)))
            }
        }
        
        case Mult(e1, e2) => {
            // YOUR CODE HERE
            if (isString(evalExpr(e1, env)) && isNatNum(evalExpr(e2, env))) {
                StringValue(valueToString(evalExpr(e1, env)) * valueToNum(evalExpr(e2, env)).toInt)
            }
            else if (isNatNum(evalExpr(e1, env)) && isString(evalExpr(e2, env))) {
                StringValue(valueToString(evalExpr(e2, env)) * valueToNum(evalExpr(e1, env)).toInt)
            }
            else if (isNum(evalExpr(e1, env)) && isNum(evalExpr(e2, env))) {
                NumValue(valueToNum(evalExpr(e1, env)).toInt * valueToNum(evalExpr(e2, env)).toInt)
            }
            else {
                throw new IllegalArgumentException("Cannot multiply string and not natural number.")
            }
        }
        case Geq(e1, e2) => {
            val v1 = valueToNum(evalExpr(e1, env))
            val v2 = valueToNum(evalExpr(e2, env))
            BoolValue(v1 >= v2)
        }
        case Not(e) => {
            val v = valueToBool(evalExpr(e, env))
            BoolValue(!v)
        }
        case IfThenElse(e, e1, e2) => {
            val v1 = valueToBool(evalExpr(e, env))
            if (v1)
                evalExpr(e1, env)
            else
                evalExpr(e2, env)
        }
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)
            val newEnv = env + (x -> v1)
            evalExpr(e2, newEnv)
        }
        
        case SubString(e, e1, e2) => {
            // YOUR CODE HERE
            val index1 = valueToNum(evalExpr(e1, env)).toInt
            val index2 = valueToNum(evalExpr(e2, env)).toInt
            StringValue(valueToString(evalExpr(e, env)).substring(index1, index2 + 1))
        }
        
    }

}

defined function evalExpr

In [42]:
// BEGIN TEST
val v1 = ConstString("hello world")
val v2 = ConstString("")
val v4 = Mult(v1, Const(20))
val v5 = Mult(v2, Const(10))
val emptyEnv: Map[String,Value] = Map()

assert(valueToString(evalExpr(v1, emptyEnv)) == "hello world", "Test 1 failed")
assert(valueToString(evalExpr(v5, emptyEnv)) == "", "Test 2 failed")
assert(valueToNum(evalExpr(StringLength(v5), emptyEnv)) == 0, "Test 3 failed")
assert(valueToNum(evalExpr(StringLength(v1), emptyEnv)) == 11, "Test 4 failed")
assert(evalExpr(StringLength(v4), emptyEnv) == NumValue(220.0), "Test 5 failed")

passed(5)
// END TEST 

*** Tests Passed (5 points) ***

v1: ConstString = ConstString("hello world")
v2: ConstString = ConstString("")
v4: Mult = Mult(ConstString(hello world),Const(20.0))
v5: Mult = Mult(ConstString(),Const(10.0))
emptyEnv: Map[String, Value] = Map()

In [43]:
// BEGIN TEST
val v1 = ConstString("hello world")
val v6 = SubString(v1,Const(0), Const(3))
val vv6 = evalExpr(v6, Map()) 
assert(vv6 == StringValue("hell"), "Test 1 failed")

val x = Identifier("x")
val v7 = Let("x", ConstString("Gangster!"), Let ("y", Const(2.0), Mult(x, Identifier("y"))))
assert(evalExpr(v7, Map()) == StringValue("Gangster!Gangster!"), "Test 2 failed")

passed(5)
// END TEST

*** Tests Passed (5 points) ***

v1: ConstString = ConstString("hello world")
v6: SubString = SubString(ConstString(hello world),Const(0.0),Const(3.0))
vv6: Value = StringValue(hell)
x: Identifier = Identifier("x")
v7: Let = Let(x,ConstString(Gangster!),Let(y,Const(2.0),Mult(Identifier(x),Identifier(y))))

## Problem 3 (15 points)

In this problem, we will explore Lettuce with function calls but with dynamic rather than static scoping. 
Let us take a bare bones subset given by the following grammar.

$$\begin{array}{rcl}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Ident(\mathbf{Identifier}) \\
& \rightarrow & Const(\mathbf{Double}) \\
& \rightarrow & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
& \rightarrow & Let(\mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) \\
& \rightarrow & \color{red}{FunDef(\mathbf{Identifier}, \mathbf{Expr})} \\
& \rightarrow & \color{red}{FunCall(\mathbf{Expr}, \mathbf{Expr})} \\
\end{array} $$

As explored earlier, we simply add new case classes to the abstract syntax tree in Scala. Next, we define the values.

In [46]:
sealed trait Program
sealed trait Expr
case class Ident(st: String) extends Expr
case class Const(f: Double) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Let (id: String, e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, bodyExpr: Expr) extends Expr
case class FunCall(e: Expr, callBody: Expr) extends Expr

defined trait Program
defined trait Expr
defined class Ident
defined class Const
defined class Plus
defined class Let
defined class FunDef
defined class FunCall

In [47]:
sealed trait Value
case class NumValue(f: Double) extends Value
case class Closure(x: String, e: Expr, sigma: Map[String, Value]) extends Value

def valueToNum(v: Value): Double = v match {
    case NumValue(f) => f
    case _ => throw new IllegalArgumentException("Asking to convert a non numeric value to number -- impossible!")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, s) => Closure(x, e, s)
    case _ => throw new IllegalArgumentException("Asking to convert a numeric value to closure -- impossible!")
}


defined trait Value
defined class NumValue
defined class Closure
defined function valueToNum
defined function valueToClosure

### 3 A (5 points): Explore static vs. dynamic scoping
Let us write the operational semantics for evaluating a function call under _dynamic scoping_. Dynamic scoping was discussed in our notebook on Lettuce.

Write down the value of the following program under **static** scoping.

~~~
let x = 10 in 
let y = 1 + x  in 
let f = function (x) 
          x * y
        in 
let y = 20 in 
    f (20)
~~~


$$
x = 20\\
y = 11\\
f(20) = 20 * 11 = 220
$$

And under **dynamic** scoping?

$$
x = 20\\
y = 20\\
f(20) = 20 * 20 = 400
$$

### Semantic rule for dynamic scoping

We will now compare semantic rules for static vs. dynamic scoping.

Note that the rule for function definitions is unchanged. 

$$ \begin{array}{c}
\\
\hline
\text{eval}(\texttt{FuncDef}(x, e), \sigma) = \text{Closure}(x, \texttt{e}, \sigma) \\
\end{array} \text{(func-def-ok)}$$

The rule for static scoping was (see the notebook titled "Function Calls in Lettuce" for an explanation).


$$ \begin{array}{c}
\text{eval}(\texttt{fun_expr}, \sigma) = \text{Closure}(\color{blue}{p}, \color{red}{\texttt{e}}, \color{green}{\pi}),\ \text{eval}(\texttt{arg_expr}, \sigma) = v_2,\ v_2 \not= \mathbf{error} \\
\hline
\texttt{eval}( \texttt{FunCall(fun_expr, arg_expr)}, \sigma )  = \texttt{eval}( \color{red}{\texttt{e}}, \color{green}{\pi} [ \color{blue}{p} \mapsto v_2 ] )\\
\end{array} \text{(funcall-static-ok)}
$$

Let us write a rule for dynamic scoping that will try to evaluate the body of the called function under the current environment. 

$$ \begin{array}{c}
\text{eval}(\texttt{fun_expr}, \sigma) = \text{Closure}(\color{blue}{p}, \color{red}{\texttt{e}}, \color{green}{\pi}),\ \text{eval}(\texttt{arg_expr}, \sigma) = v_2,\ v_2 \not= \mathbf{error} \\
\hline
\texttt{eval}( \texttt{FunCall(fun_expr, arg_expr)}, \sigma )  = \texttt{eval}( \color{red}{\texttt{e}}, \sigma[p \mapsto v_2] )\\
\end{array} \text{(funcall-dynamic-ok)}
$$


### 3 B (10 points): Write an interpreter

Let us now write an interpreter that will implement dynamic scoping for function calls. Fill in missing portions from below.

In [56]:
def evalExprDynamic(e: Expr, env: Map[String, Value]): Value =  e match {
    case Ident(x) => { 
        if (env contains x) {
            env(x)
        } else
            throw new IllegalArgumentException(s"Identifier $x not found")
    }
    
    case Const(f) => NumValue(f)
    case Plus(e1, e2) => {
        val v1 = valueToNum(evalExprDynamic(e1, env))
        val v2 = valueToNum(evalExprDynamic(e2, env))
        NumValue(v1 + v2)
    }
    case Let(x, e1, e2) => {
        val v1 = evalExprDynamic(e1, env)
        val newEnv = env + (x -> v1)
        evalExprDynamic(e2, newEnv)
    }
    
    case FunDef(id, e) => {
        Closure(id, e, env)
    }
    
    case FunCall(callExpr, argExpr) => { 
        // YOUR CODE HERE
        val eval1 = evalExprDynamic(callExpr, env)
        val eval2 = evalExprDynamic(argExpr, env)
        eval1 match {
            case Closure(p, e, pi) => {
                val newEnv = env + (p -> eval2)
                evalExprDynamic(e, newEnv)
            }
        }
    }
}

defined function evalExprDynamic

In [57]:
//BEGIN TEST

// let x = 10 in let f = function (y) y + x in let x = 11 in f(11)
val x = Ident("x")
val y = Ident("y")
val f = Ident("f")
val e1 = Let("x", Const(11), FunCall(f, Const(11)))
val e2 = Let("f", FunDef("y", Plus(y, x)), e1)
val e3 = Let("x", Const(10), e2)

assert( evalExprDynamic(e3, Map()) == NumValue(22.0), "Test 1 failed" )

passed(4)
//END TEST

*** Tests Passed (4 points) ***

x: Ident = Ident("x")
y: Ident = Ident("y")
f: Ident = Ident("f")
e1: Let = Let(x,Const(11.0),FunCall(Ident(f),Const(11.0)))
e2: Let = Let(f,FunDef(y,Plus(Ident(y),Ident(x))),Let(x,Const(11.0),FunCall(Ident(f),Const(11.0))))
e3: Let = Let(x,Const(10.0),Let(f,FunDef(y,Plus(Ident(y),Ident(x))),Let(x,Const(11.0),FunCall(Ident(f),Const(11.0)))))

In [58]:
//BEGIN TEST
/*
let x = 10 in 
let y = 1 + x  in 
let f = function (x) 
          x + y
        in 
let y = 20 in 
    f (20)
    */

val x = Ident("x")
val y = Ident("y")
val f = Ident("f")
val e1 = Let("y", Const(20), FunCall(f, Const(20)))
val e2 = Let("f", FunDef("x", Plus(y, x)), e1)
val e3 = Let("y", Plus(Const(1), x), e2)
val e4 = Let("x", Const(10), e3)

assert( evalExprDynamic(e4, Map()) == NumValue(40.0), "Test 2 failed" )

passed(3)
//END TEST

*** Tests Passed (3 points) ***

x: Ident = Ident("x")
y: Ident = Ident("y")
f: Ident = Ident("f")
e1: Let = Let(y,Const(20.0),FunCall(Ident(f),Const(20.0)))
e2: Let = Let(f,FunDef(x,Plus(Ident(y),Ident(x))),Let(y,Const(20.0),FunCall(Ident(f),Const(20.0))))
e3: Let = Let(y,Plus(Const(1.0),Ident(x)),Let(f,FunDef(x,Plus(Ident(y),Ident(x))),Let(y,Const(20.0),FunCall(Ident(f),Const(20.0)))))
e4: Let = Let(x,Const(10.0),Let(y,Plus(Const(1.0),Ident(x)),Let(f,FunDef(x,Plus(Ident(y),Ident(x))),Let(y,Const(20.0),FunCall(Ident(f),Const(20.0))))))

In [59]:
//BEGIN TEST
/*
let x = 10 in 
let y = 1 + x  in 
let z = 1 + y in 
let g = function (y)
        x + y 
        in 
let f = function (a) 
         g(a + x)
        in 
let x = 20 in 
    f (y)
    */

val x = Ident("x")
val y = Ident("y")
val f = Ident("f")
val z = Ident("z")
val w = Ident("w")
val a = Ident("a")

val e1 = Let("x", Const(20), FunCall(f, y)) // let y =  20 in f(y)(30)
val fn_f = FunDef("a", FunCall(Ident("g"), Plus(a, x))) // function(a) g(a + x)
val e2 = Let("f", fn_f, e1) // let f = function(a) ..
val fn_g = FunDef("y", Plus(x, y)) 
val e3 = Let("g", fn_g, e2)
val e4 = Let("z", Plus(Const(1), y), e3) // let z = 1 + y in ...
val e5 = Let("y", Plus(Const(1), x), e4) // let y = 1 + x in ...
val e6 = Let("x", Const(10), e5)// let x = 10 in ..
assert( evalExprDynamic(e6, Map()) == NumValue(51.0), "Test 3 failed")

passed(3)
//END TEST

*** Tests Passed (3 points) ***

x: Ident = Ident("x")
y: Ident = Ident("y")
f: Ident = Ident("f")
z: Ident = Ident("z")
w: Ident = Ident("w")
a: Ident = Ident("a")
e1: Let = Let(x,Const(20.0),FunCall(Ident(f),Ident(y)))
fn_f: FunDef = FunDef(a,FunCall(Ident(g),Plus(Ident(a),Ident(x))))
e2: Let = Let(f,FunDef(a,FunCall(Ident(g),Plus(Ident(a),Ident(x)))),Let(x,Const(20.0),FunCall(Ident(f),Ident(y))))
fn_g: FunDef = FunDef(y,Plus(Ident(x),Ident(y)))
e3: Let = Let(g,FunDef(y,Plus(Ident(x),Ident(y))),Let(f,FunDef(a,FunCall(Ident(g),Plus(Ident(a),Ident(x)))),Let(x,Const(20.0),FunCall(Ident(f),Ident(y)))))
e4: Let = Let(z,Plus(Const(1.0),Ident(y)),Let(g,FunDef(y,Plus(Ident(x),Ident(y))),Let(f,FunDef(a,FunCall(Ident(g),Plus(Ident(a),Ident(x)))),Let(x,Const(20.0),FunCall(Ident(f),Ident(y))))))
e5: Let = Let(y,Plus(Const(1.0),Ident(x)),Let(z,Plus(Const(1.0),Ident(y)),Let(g,FunDef(y,Plus(Ident(x),Ident(y))),Let(f,FunDef(a,FunCall(Ident(g),Plus(Ident(a),Ident(x)))),Let(x,Const(20.0),FunCall(Ident(f),Ident(y)))))))
e6: Let = Let(

## 3 (D) Challenge Problem (Do not submit with assignment)

Let us investigate the behavior of the following program under the dynamic scoping rule thus far.

~~~
let x = 10 in 
let y = 1 + x in 
let z = 1 + y in 
let f = function (x) function (w)
          x + y + z + w
        in
let y = 20 in 
    f(y)(30)
~~~

What should the value of this function be under dynamic scoping? To help you out, here is the same program in python that implements dynamic scoping.

~~~
x = 10
y = x + 1
z = y + 1
def f(x):
   def g(w):
       return x + y + z + w
   return g
y = 20
f(y)(30)
~~~

However, check out what our interpreter returns:

In [24]:
val x = Ident("x")
val y = Ident("y")
val f = Ident("f")
val z = Ident("z")
val w = Ident("w")

val e1 = Let("y", Const(20), FunCall(FunCall(f, y), Const(30) )) // let y = 20 in f(y)(30)
val innerFn = FunDef("w", Plus( Plus(x,y), Plus(z,w)))
val e2 = Let("f", FunDef("x", innerFn), e1)
val e3 = Let("z", Plus(Const(1), y), e2)
val e4 = Let("y", Plus(Const(1), x), e3)
val e5 = Let("x", Const(10), e4)

print(s"my interpreter returns: ${evalExprDynamic(e5, Map())}")

: 

Explain why the result is wrong/different from what dynamic scoping in Python produces? Fix the interpreter to handle dynamic scoping correctly. Email your answer (code and explanation) directly to Sriram. First two answers will get a prize. The challenge problem will be due in 2 weeks.